In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from time import time
%matplotlib inline

In [2]:
train = pd.read_csv('all/train.csv',parse_dates=[2])
test = pd.read_csv('all/test.csv',parse_dates=[3])
store = pd.read_csv('all/store.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test.fillna(1, inplace=True)
store.fillna(0, inplace=True)

In [4]:
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

In [5]:
train = train.sort_values(['Date'],ascending = False)
ho_val = train[:6*7*1115]
ho_train = train[6*7*1115:]

In [6]:
ho_val = ho_val[ho_val["Open"] != 0]
ho_val = ho_val[ho_val["Sales"] > 0]
ho_train = ho_train[ho_train["Open"] != 0]
ho_train = ho_train[ho_train["Sales"] > 0]


In [7]:
from feature_engineering import features_create

In [8]:
features_create(ho_train)
features_create(ho_val)
features_create(test)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Promo2SinceYear,PromoInterval,Year,Month,Day,WeekOfYear,CompetitionOpen,PromoOpen,monthStr,IsPromoMonth
0,1,1,3,2015-09-17,1.0,1,0,0,3,1,...,0.0,,2015,9,17,38,84.0,24189.50,Sept,0
1,857,1,2,2015-09-16,1.0,1,0,0,3,1,...,0.0,,2015,9,16,38,84.0,24189.50,Sept,0
2,1713,1,1,2015-09-15,1.0,1,0,0,3,1,...,0.0,,2015,9,15,38,84.0,24189.50,Sept,0
3,2569,1,0,2015-09-14,1.0,1,0,0,3,1,...,0.0,,2015,9,14,38,84.0,24189.50,Sept,0
4,3425,1,6,2015-09-13,0.0,0,0,0,3,1,...,0.0,,2015,9,13,37,84.0,24189.25,Sept,0
5,4281,1,5,2015-09-12,1.0,0,0,0,3,1,...,0.0,,2015,9,12,37,84.0,24189.25,Sept,0
6,5137,1,4,2015-09-11,1.0,0,0,0,3,1,...,0.0,,2015,9,11,37,84.0,24189.25,Sept,0
7,5993,1,3,2015-09-10,1.0,0,0,0,3,1,...,0.0,,2015,9,10,37,84.0,24189.25,Sept,0
8,6849,1,2,2015-09-09,1.0,0,0,0,3,1,...,0.0,,2015,9,9,37,84.0,24189.25,Sept,0
9,7705,1,1,2015-09-08,1.0,0,0,0,3,1,...,0.0,,2015,9,8,37,84.0,24189.25,Sept,0


In [9]:
ho_train.drop(['Date','Customers','Open','PromoInterval','monthStr'],axis=1,inplace =True)
ho_val.drop(['Date','Customers','Open','PromoInterval','monthStr'],axis=1,inplace =True)


In [10]:
xtest =test.drop(['Id','Date','Open','PromoInterval','monthStr'],axis = 1)

In [11]:
ho_xtrain = ho_train.drop(['Sales'],axis=1 )
ho_ytrain = np.log1p(ho_train.Sales)
ho_xval = ho_val.drop(['Sales'],axis=1 )
ho_yval = np.log1p(ho_val.Sales)


In [12]:
def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)

In [28]:
import xgboost as xgb
params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.03,
          "max_depth": 10,
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "silent": 1,
          "seed": 10
          }
num_boost_round = 300


dtrain = xgb.DMatrix(ho_xtrain, ho_ytrain)
dvalid = xgb.DMatrix(ho_xval, ho_yval)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]


In [ ]:
print("Train a XGBoost model")
start = time()
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, 
  early_stopping_rounds=100, feval=rmspe_xg, verbose_eval=True)
end = time()
print('Training time is {:2f} s.'.format(end-start))


Train a XGBoost model
[0]	train-rmse:8.02088	eval-rmse:8.02723	train-rmspe:0.999808	eval-rmspe:0.99981
Multiple eval metrics have been passed: 'eval-rmspe' will be used for early stopping.

Will train until eval-rmspe hasn't improved in 100 rounds.
[1]	train-rmse:7.78072	eval-rmse:7.78748	train-rmspe:0.99971	eval-rmspe:0.999714
[2]	train-rmse:7.54777	eval-rmse:7.555	train-rmspe:0.99959	eval-rmspe:0.999595
[3]	train-rmse:7.3218	eval-rmse:7.32953	train-rmspe:0.999444	eval-rmspe:0.999451
[4]	train-rmse:7.10262	eval-rmse:7.1102	train-rmspe:0.999267	eval-rmspe:0.999276
[5]	train-rmse:6.89009	eval-rmse:6.89751	train-rmspe:0.999054	eval-rmspe:0.999066
[6]	train-rmse:6.68389	eval-rmse:6.69188	train-rmspe:0.9988	eval-rmspe:0.998816
[7]	train-rmse:6.48396	eval-rmse:6.49139	train-rmspe:0.998498	eval-rmspe:0.998517
[8]	train-rmse:6.28996	eval-rmse:6.298	train-rmspe:0.998143	eval-rmspe:0.998167
[9]	train-rmse:6.10183	eval-rmse:6.10968	train-rmspe:0.997725	eval-rmspe:0.997754
[10]	train-rmse:5.91932